# Preparación de datos MLOps (_todas las categorías)

Este notebook permite transformar los datos a series temporales, filtrando por una categoría concreta (por ejemplo, 'BOLLERIA') o mostrando todas las categorías si no se especifica ninguna.

In [11]:
import sys
import importlib
# Recargar el módulo data_utils_02 para asegurar que se carguen los cambios
if 'src.data_utils_02' in sys.modules:
    importlib.reload(sys.modules['src.data_utils_02'])

## Carga de datos

Este notebook encapsula el proceso de carga de datos, transformación a series temporales y creación de características y target en funciones reutilizables para forecasting.

In [13]:
from src.data_utils_02 import load_raw_data 
# Descarga y carga datos raw desde BigQuery para el rango de fechas especificado
df_raw = load_raw_data(
    fecha_inicio="2023-01-02", 
    fecha_fin="2025-06-30", 
    descargar_bq=False)
df_raw.describe()

,fecha,cantidad,base_imponible,tipo_IVA,total,is_summer_peak,is_easter
count,337353,337353.000000,337353.000000,335106.000000,337353.000000,337353.000000,337353.0
mean,2024-03-10 00:54:30.032281344,1.209303,2.010886,9.084558,2.199180,0.208473,0.0
min,2023-01-02 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,2023-07-28 00:00:00,1.000000,1.350000,10.000000,1.450000,0.000000,0.0
50%,2024-03-09 00:00:00,1.000000,1.640000,10.000000,1.800000,0.000000,0.0
75%,2024-09-25 00:00:00,1.000000,2.320000,10.000000,2.550000,0.000000,0.0
max,2025-06-30 00:00:00,93.000000,287.270000,10.000000,316.000000,1.000000,0.0
std,NaN,0.733263,1.692278,2.793367,1.862156,0.406217,0.0


## Transformación a series temporales por categoría

In [36]:
from src.data_utils_02 import transformar_a_series_temporales, guardar_time_series_interim # type: ignore

# Selecciona la categoría (por ejemplo, 'BOLLERIA'). Si es None, se muestran todas las categorías.
categoria = None  # Cambia a None (sin comillas) para mostrar todas las categorías

if categoria is not None and isinstance(categoria, str) and categoria != '':
    df_filtrado = df_raw[df_raw['familia'] == categoria]
    print(f'Filtrando por categoría: {categoria}')
else:
    df_filtrado = df_raw
    print('Mostrando todas las categorías')

df_familia_semanal = transformar_a_series_temporales(
    df_filtrado, 
    familia=categoria, 
    guardar_interim=True
)
df_familia_semanal.head(150)

Mostrando todas las categorías
Archivo guardado en: C:\Workspace\mlops_fleca_project\data\interim\time_series_TODAS_weekly_20250804.parquet


,year,week,base_imponible,is_summer_peak,is_easter,dias_semana,week_start
0,2023,1,5200.02,0,0,7,2023-01-02
1,2023,2,4020.09,0,0,7,2023-01-09
2,2023,3,4406.36,0,0,7,2023-01-16
3,2023,4,4022.09,0,0,7,2023-01-23
4,2023,5,3952.34,0,0,7,2023-01-30
...,...,...,...,...,...,...,...
121,2025,22,5300.39,0,0,7,2025-05-26
122,2025,23,5296.86,0,0,7,2025-06-02
123,2025,24,6010.64,0,0,7,2025-06-09
124,2025,25,5705.66,0,0,7,2025-06-16


In [37]:
# Verificar los valores de is_easter en las fechas de Semana Santa
easter_dates = df_familia_semanal[df_familia_semanal['is_easter'] > 0]
print("Fechas con is_easter > 0:")
print(easter_dates[['year', 'week', 'is_easter']])

# Verificar que tenemos las 3 semanas de Semana Santa correctas
expected_easter_weeks = [
    (2023, 14),  # Semana Santa 2023
    (2024, 13),  # Semana Santa 2024
    (2025, 16)   # Semana Santa 2025
]

for year, week in expected_easter_weeks:
    is_marked = ((df_familia_semanal['year'] == year) & 
                 (df_familia_semanal['week'] == week) & 
                 (df_familia_semanal['is_easter'] == 1)).any()
    print(f"Semana {week} del {year} marcada como Semana Santa: {is_marked}")

Fechas con is_easter > 0:
     year  week  is_easter
13   2023    14          1
62   2024    13          1
115  2025    16          1
Semana 14 del 2023 marcada como Semana Santa: True
Semana 13 del 2024 marcada como Semana Santa: True
Semana 16 del 2025 marcada como Semana Santa: True


In [38]:
df_familia_semanal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   year            126 non-null    UInt32        
 1   week            126 non-null    UInt32        
 2   base_imponible  126 non-null    float64       
 3   is_summer_peak  126 non-null    int64         
 4   is_easter       126 non-null    int64         
 5   dias_semana     126 non-null    int64         
 6   week_start      126 non-null    datetime64[ns]
dtypes: UInt32(2), datetime64[ns](1), float64(1), int64(3)
memory usage: 6.3 KB


## Transformación a features y target

In [40]:
from src.data_utils_02 import transformar_features_target, guardar_datos_procesados # type: ignore
# Generación de features y target para modelado
X, y, df_completo = transformar_features_target(
    df_familia_semanal,
    lags_list=[1, 2, 3, 52],
    columna_target='base_imponible',
    cols_exogenas=['is_summer_peak', 'is_easter'],
    eliminar_nulos=True
)
print(f'Dimensión de X (features): {X.shape}')
print(f'Dimensión de y (target): {y.shape}')
print('Primeras filas de X:')
X.head(200)

Dimensión de X (features): (73, 7)
Dimensión de y (target): (73,)
Primeras filas de X:


,base_imponible_lag1,base_imponible_lag2,base_imponible_lag3,base_imponible_lag52,is_summer_peak,is_easter,week_start
52,3578.16,3240.28,3434.63,5200.02,0,0,2024-01-15
53,3790.88,3578.16,3240.28,4020.09,0,0,2024-01-22
54,4096.18,3790.88,3578.16,4406.36,0,0,2024-01-29
55,3750.64,4096.18,3790.88,4022.09,0,0,2024-02-05
56,3977.76,3750.64,4096.18,3952.34,0,0,2024-02-12
...,...,...,...,...,...,...,...
120,5118.78,4969.73,5165.66,5599.05,0,0,2025-05-19
121,4464.67,5118.78,4969.73,5305.38,0,0,2025-05-26
122,5300.39,4464.67,5118.78,4844.84,0,0,2025-06-02
123,5296.86,5300.39,4464.67,5107.76,0,0,2025-06-09


In [41]:
print(df_completo.columns.tolist())

['base_imponible', 'is_summer_peak', 'is_easter', 'week_start', 'base_imponible_lag1', 'base_imponible_lag2', 'base_imponible_lag3', 'base_imponible_lag52', 'base_imponible_next1']


In [42]:
# Guardar los datasets en la carpeta processed
archivos_guardados = guardar_datos_procesados(
    X=X, 
    y=y, 
    df_completo=df_completo,
    familia=categoria if categoria else 'TODAS'
)
for tipo_datos, ruta in archivos_guardados.items():
    print(f'Archivo {tipo_datos}: {ruta}')

Datos procesados guardados en la carpeta: C:\Workspace\mlops_fleca_project\data\processed
- Features (X): ts_X_todas_20250804.parquet
- Target (y): ts_y_todas_20250804.parquet
- Dataset completo: ts_df_todas_20250804.parquet
Archivo X: C:\Workspace\mlops_fleca_project\data\processed\ts_X_todas_20250804.parquet
Archivo y: C:\Workspace\mlops_fleca_project\data\processed\ts_y_todas_20250804.parquet
Archivo df_completo: C:\Workspace\mlops_fleca_project\data\processed\ts_df_todas_20250804.parquet
